### Deep Learning Homework 6

Taking inspiration from the last 2 pictures within the notebook (07-convnets.ipynb), implement a U-Net-style CNN with the following specs:

1. All convolutions must use a 3 x 3 kernel and leave the spatial dimensions (i.e. height, width) of the input untouched.
2. Downsampling in the contracting part is performed via maxpooling with a 2 x 2 kernel and stride of 2.
3. Upsampling is operated by a deconvolution with a 2 x 2 kernel and stride of 2. The PyTorch module that implements the deconvolution is `nn.ConvTranspose2d`
4. The final layer of the expanding part has only 1 channel 

* between how many classes are we discriminating?

Create a network class with (at least) a `__init__` and a `forward` method. Please resort to additional structures (e.g., `nn.Module`s, private methods...) if you believe it helps readability of your code.

Test, at least with random data, that the network is doing the correct tensor operations and that the output has the correct shape (e.g., use `assert`s in your code to see if the byproduct is of the expected shape).

Note: the overall organization of your work can greatly improve readability and understanding of your code by others. Please consider preparing your notebook in an organized fashion so that we can better understand (and correct) your implementation.

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pylab as pl
from IPython.display import clear_output

In [40]:
x = torch.zeros((1, 3, 324, 200))

nn.ConvTranspose2d(3, 3, kernel_size=2, stride=2)(x).size()


torch.Size([1, 3, 648, 400])

In [ ]:
# Define a model
class U_net(nn.Module):
    """
    Implements a Unet
    """
    
    def VGG_block(self, in_channels, out_channels, downsample=False, return_raw=False):
        new_layers = []
        new_layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        new_layers.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
        if downsample:
            new_layers.append(nn.MaxPool2d(2))
            
        if return_raw:
            return new_layers
        
        return nn.Sequential(*new_layers)
    
    def upsampling_block(self, in_channels, out_channels, return_raw=False):
        new_layers = self.VGG_block(in_channels, out_channels, return_raw=True)
        new_layers.append(nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2))
        
        if return_raw:
            return new_layers
        
        return nn.Sequential(*new_layers)
            
        
    def __init__(self, h=572, w=572, channels=3, depth=4, n_classes=10):
        super().__init__()
        
        # Downsampling layers
        self.downsampling_layers = []
        in_channels, out_channels = channels, 64
        for i in range(depth):
            self.downsampling_layers.append(self.VGG_block(in_channels, out_channels))
            in_channels = out_channels
            out_channels *= 2
        
        # Deepest layer
        self.deep_layer = self.VGG_block(in_channels, out_channels)
        
        # Upsampling layers
        self.upsampling_layers = []
        out_channels, in_channels = in_channels, out_channels
        for i in range(depth):
            self.upsampling_layers.append(self.upsampling_block(in_channels, out_channels))
            in_channels = out_channels
            out_channels = out_channels // 2

    def forward(self, X):        
        return self.layers(X)